<a href="https://colab.research.google.com/github/JuanZapa7a/semiotics/blob/main/QAT_MNIST_NOISY_CMOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantization Aware Training (QAT) using Larq for a noisy CMOS binarized quantization with the CIFAR10 dataset

## 1. Installation of Larq and necessary dependencies

In [2]:
!pip -q install tensorflow==2.10.0
!pip -q install larq==0.13.1

import tensorflow as tf
import larq as lq

## 2. Data preparation (MNIST)

Download and prepare the MNIST dataset.

By default, each MNIST image has a shape of (28, 28), which is 2D.
However, neural networks (especially convolutional networks) typically expect 3D inputs: (height, width, channels).
Adding a channel dimension (with value 1 for grayscale) changes each image shape from (28, 28) to (28, 28, 1), which is required for most neural network layers to interpret the images correctly.
The overall shapes for the dataset become (60000, 28, 28, 1) for training and (10000, 28, 28, 1) for testing.

The MNIST dataset’s pixel values originally range from 0 to 255.
Dividing by 127.5 and then subtracting 1 maps the values to a -1 to 1 range, which can help certain models converge more quickly and maintain numerical stability. (Centering pixel values around zero often benefits neural networks as it reduces bias and helps gradient-based methods perform better.)

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) # (60000, 28, 28) (60000,)
test_images = test_images.reshape((10000, 28, 28, 1)) # (10000, 28, 28) (10000,)

# For binarized models, it is standard to normalize images to a range between -1 and 1.
train_images, test_images = train_images / 127.5 - 1,test_images / 127.5 - 1

print(train_images.shape, train_labels.shape)  # Debe ser (60000, 28, 28, 1), (60000,)
print(test_images.shape, test_labels.shape)    # Debe ser (10000, 28, 28, 1), (10000,)

11490434/11490434 [==============================] - 2s 0us/step
(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


## 3. AÑADIENDO RUIDO A LOS PESOS Y A LAS ACTIVACIONES

Para modificar el código y permitir que el ruido se inyecte tanto en los pesos como en las activaciones, podemos expandir la lógica de la clase `NoisyLayer`. Además, sí, las imágenes de entrada podrían venir con ruido (simulando ruido real del sensor o del ambiente), lo que podría mejorarse añadiendo ruido a las activaciones de cada capa.

### **Código modificado para añadir ruido  a los pesos y activaciones:**

1. **Inyectar ruido en las activaciones**:
    - Modifica la función `call` de `NoisyLayer` para agregar ruido a las activaciones después de aplicar la lógica de la capa base.
2. **Simular ruido en la entrada (opcional)**:
    - Añade una capa adicional de ruido a las imágenes de entrada con `tf.keras.layers.GaussianNoise`.

### **Explicación de los cambios**

#### 1. **Ruido en activaciones**

- Después de aplicar la lógica de la capa base (`self.wrapped_layer(inputs)`), se genera ruido para las activaciones con:
    
    ```python
    activation_noise = tf.random.normal(
        shape=tf.shape(outputs),
        mean=self.noise_mean,
        stddev=self.noise_stddev_activations
    )
    outputs_noisy = outputs + activation_noise
    ```
    
- Esto inyecta ruido gaussiano en las salidas de cada capa.

#### 2. **Simulación de imágenes de entrada con ruido**

- La capa `tf.keras.layers.GaussianNoise` añade ruido gaussiano directamente a las imágenes de entrada:
    
    ```python
    model.add(tf.keras.layers.GaussianNoise(0.05, input_shape=(28, 28, 1)))
    ```
    
- Esto simula el ruido que podría provenir de sensores o entornos del mundo real.

#### 3. **Parámetros separados para ruido en pesos y activaciones**

- `noise_stddev_weights`: Desviación estándar para el ruido en los pesos.
- `noise_stddev_activations`: Desviación estándar para el ruido en las activaciones.

---

### **Escenarios del ruido en imágenes**

La simulación de ruido en las imágenes puede ser útil para:

1. **Robustez a entradas ruidosas**:
    - Por ejemplo, cámaras con baja iluminación, sensores defectuosos, o datos transmitidos con ruido.
2. **Regularización adicional**:
    - Similar a `Dropout`, añade variabilidad a las entradas, evitando que el modelo dependa de patrones específicos.

---

### **Resultados esperados**

Con este enfoque:

- El modelo será más robusto a perturbaciones tanto en las entradas como en las representaciones internas (activaciones y pesos).
- Se puede usar para estudiar cómo el ruido afecta al rendimiento en diferentes partes del pipeline de entrenamiento.

In [4]:
import larq as lq
import tensorflow as tf

# Define a custom layer to add noise to weights and activations
class NoisyLayer(tf.keras.layers.Layer):
    def __init__(self, wrapped_layer, noise_mean=0.0, noise_stddev_weights=0.01, noise_stddev_activations=0.01, **kwargs):
        super().__init__(**kwargs)
        self.wrapped_layer = wrapped_layer                 # Original layer to wrap
        self.noise_mean = noise_mean                       # Mean of the noise
        self.noise_stddev_weights = noise_stddev_weights   # Stddev of noise for weights
        self.noise_stddev_activations = noise_stddev_activations  # Stddev of noise for activations

    def build(self, input_shape):
        # Build the wrapped layer
        self.wrapped_layer.build(input_shape)
        super().build(input_shape)

    def call(self, inputs):
        # Add noise to weights
        for weight in self.wrapped_layer.trainable_weights:
            noise = tf.random.normal(
                shape=tf.shape(weight),
                mean=self.noise_mean,
                stddev=self.noise_stddev_weights
            )
            noisy_weight = weight + noise  # Add noise to weights
            weight.assign(noisy_weight)   # Assign the noisy weights temporarily

        # Forward pass through the wrapped layer
        outputs = self.wrapped_layer(inputs)

        # Add noise to activations
        activation_noise = tf.random.normal(
            shape=tf.shape(outputs),
            mean=self.noise_mean,
            stddev=self.noise_stddev_activations
        )
        outputs_noisy = outputs + activation_noise

        return outputs_noisy

# Define default quantization options for quantized layers
kwargs = dict(
    input_quantizer="ste_sign",
    kernel_quantizer="ste_sign",
    kernel_constraint="weight_clip"
)

# Initialize a Sequential model
model_wa = tf.keras.models.Sequential()

# Add noise to input images (simulate noisy input images)
model_wa.add(tf.keras.layers.GaussianNoise(0.05, input_shape=(28, 28, 1)))

# First convolutional layer with noise on weights and activations
model_wa.add(NoisyLayer(
    lq.layers.QuantConv2D(
        32, (3, 3),
        kernel_quantizer="ste_sign",
        kernel_constraint="weight_clip",
        use_bias=False
    ),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01
))
model_wa.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_wa.add(tf.keras.layers.BatchNormalization(scale=False))

# Second convolutional layer with noise on weights and activations
model_wa.add(NoisyLayer(
    lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01
))
model_wa.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_wa.add(tf.keras.layers.BatchNormalization(scale=False))

# Third convolutional layer with noise on weights and activations
model_wa.add(NoisyLayer(
    lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01
))
model_wa.add(tf.keras.layers.BatchNormalization(scale=False))
model_wa.add(tf.keras.layers.Flatten())

# Dense layer with noise on weights and activations
model_wa.add(NoisyLayer(
    lq.layers.QuantDense(64, use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01
))
model_wa.add(tf.keras.layers.BatchNormalization(scale=False))

# Output layer with noise on weights and activations
model_wa.add(NoisyLayer(
    lq.layers.QuantDense(10, use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01
))
model_wa.add(tf.keras.layers.BatchNormalization(scale=False))
model_wa.add(tf.keras.layers.Activation("softmax"))

# Especificar la forma de entrada (28, 28, 1) para imágenes MNIST
model_wa.build((None, 28, 28, 1))

# Summarize the model with Larq
lq.models.summary(model_wa)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x7e219b562620>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x7e219b562620>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
+sequential stats-----------------------------------------------------------------+
| Layer                  Input prec.           Outputs  # 1-bit  # 32-bit  Memory |
|                              (bit)                        x 1       x 1    (kB) |
+---------------------------------------------------------------------------------+
| gaussian_noise                   -   (-1, 28, 28, 1)        0         0       0 |
| noisy_layer                      -  (-1, 26, 26, 32)      288         0    0.04 |
| max_pooling2d                    -  (-1, 13, 13, 32)        0         0       0 |
| batch_normalization              -  (-1, 13, 13, 32)        0        64    0.25 |
| noisy_layer_1          

## Explicación de la salida

Esta salida proporciona un resumen detallado de las características del modelo secuencial con cuantización y las estadísticas asociadas. Aquí está una explicación paso a paso de cada sección:

---

### **1. Tabla de capas con estadísticas (`+sequential stats+`)**
#### **Columnas principales:**
- **Layer**: Nombre de la capa en el modelo.
- **Input prec. (bit)**: Precisión de los datos de entrada (no especificada aquí porque varía).
- **Outputs**: Dimensión de la salida de la capa.
- **# 1-bit**: Número de pesos binarios (1-bit) en la capa.
- **# 32-bit**: Número de parámetros de 32 bits (normalmente de Batch Normalization).
- **Memory (kB)**: Memoria requerida para almacenar los parámetros de la capa.

#### **Detalles por capa:**
1. **`gaussian_noise`**:
   - Agrega ruido gaussiano al input. No tiene parámetros ni memoria asignada (solo afecta durante el entrenamiento).

2. **`noisy_layer`**:
   - La primera capa convolucional cuantizada con 32 filtros de 3x3:
     - **288 pesos binarios (1-bit)**, calculados como: $ 3 \times 3 \times 1 \times 32 = 288 $.
     - **0 parámetros de 32 bits** porque la capa no usa sesgo.
     - Memoria: **0.04 kB** (288 bits = 36 bytes = 0.04 kB).

3. **`max_pooling2d`**:
   - Reduce la dimensión espacial de la salida sin usar parámetros.

4. **`batch_normalization`**:
   - Normaliza la salida del pooling:
     - **64 parámetros de 32 bits** (2 por filtro: media y desviación estándar).
     - Memoria: **0.25 kB**.

5. **`noisy_layer_1`**:
   - Segunda capa convolucional con 64 filtros de 3x3:
     - **18432 pesos binarios (1-bit)**, calculados como: $ 3 \times 3 \times 32 \times 64 = 18432 $.
     - Memoria: **2.25 kB**.

6. **`batch_normalization_1`**:
   - Igual que el anterior, pero ahora con **128 parámetros de 32 bits** porque hay 64 filtros.

7. **`noisy_layer_2`**:
   - Tercera capa convolucional:
     - **36864 pesos binarios (1-bit)**, calculados como: $ 3 \times 3 \times 64 \times 64 = 36864 $.
     - Memoria: **4.50 kB**.

8. **`flatten`**:
   - Aplana la salida para conectarla a capas densas. No tiene parámetros.

9. **`noisy_layer_3`**:
   - Capa densa con 64 unidades:
     - **36864 pesos binarios (1-bit)**, calculados como: $ 576 \times 64 = 36864 $.
     - Memoria: **4.50 kB**.

10. **`noisy_layer_4`**:
    - Capa de salida densa con 10 unidades:
      - **640 pesos binarios (1-bit)**, calculados como: $ 64 \times 10 = 640 $.
      - Memoria: **0.08 kB**.

11. **`activation`**:
    - Usa una función `softmax`. No tiene parámetros.

---

### **2. Resumen general (`+sequential summary+`)**
#### **Estadísticas importantes:**
- **Total params**: Número total de parámetros:
  - **93.6 k**: Representa 93088 pesos binarios (1-bit) y 468 parámetros de 32 bits.

- **Trainable params**: Parámetros ajustables durante el entrenamiento:
  - **93.1 k**, que incluye los pesos binarios (1-bit) y parámetros de Batch Normalization.

- **Non-trainable params**: Parámetros que no se ajustan (e.g., estadísticas acumuladas en Batch Normalization):
  - **468**.

- **Model size**:
  - **13.19 KiB**: Tamaño total del modelo en memoria comprimida (pesos binarios).
  - **Model size (8-bit FP weights)**: Si se almacenaran en precisión de 8 bits, ocuparían **11.82 KiB**.
  - **Float-32 Equivalent**: Si todos los pesos fueran flotantes de 32 bits, el modelo ocuparía **365.45 KiB**.

- **Compression Ratio of Memory**:
  - **0.04**: Indica que el modelo es 25 veces más pequeño (4%) en comparación con su equivalente en punto flotante de 32 bits.

- **Number of MACs**:
  - **0**: Esto puede deberse a que el modelo usa operaciones de conteo de bits (binarias) en lugar de acumulaciones/multiplicaciones tradicionales (MAC).

---

### **Conclusión**
La salida refleja que este modelo es **altamente eficiente en términos de memoria**, gracias al uso de **cuantización binaria** para los pesos. Aunque esto reduce la precisión de los cálculos, el modelo sigue siendo compacto y adecuado para dispositivos con recursos limitados, como hardware IoT o ASICs personalizados.

---

### **1. Tabla de capas con estadísticas (`+sequential stats+`)**
#### **Columnas principales:**
- **Layer**: Nombre de la capa en el modelo.
- **Input prec. (bit)**: Precisión de los datos de entrada (no especificada aquí porque varía).
- **Outputs**: Dimensión de la salida de la capa.
- **# 1-bit**: Número de pesos binarios (1-bit) en la capa.
- **# 32-bit**: Número de parámetros de 32 bits (normalmente de Batch Normalization).
- **Memory (kB)**: Memoria requerida para almacenar los parámetros de la capa.

#### **Detalles por capa:**
1. **`gaussian_noise`**:
   - Agrega ruido gaussiano al input. No tiene parámetros ni memoria asignada (solo afecta durante el entrenamiento).

2. **`noisy_layer`**:
   - La primera capa convolucional cuantizada con 32 filtros de 3x3:
     - **288 pesos binarios (1-bit)**, calculados como: $ 3 \times 3 \times 1 \times 32 = 288 $.
     - **0 parámetros de 32 bits** porque la capa no usa sesgo.
     - Memoria: **0.04 kB** (288 bits = 36 bytes = 0.04 kB).

3. **`max_pooling2d`**:
   - Reduce la dimensión espacial de la salida sin usar parámetros.

4. **`batch_normalization`**:
   - Normaliza la salida del pooling:
     - **64 parámetros de 32 bits** (2 por filtro: media y desviación estándar).
     - Memoria: **0.25 kB**.

5. **`noisy_layer_1`**:
   - Segunda capa convolucional con 64 filtros de 3x3:
     - **18432 pesos binarios (1-bit)**, calculados como: $ 3 \times 3 \times 32 \times 64 = 18432 $.
     - Memoria: **2.25 kB**.

6. **`batch_normalization_1`**:
   - Igual que el anterior, pero ahora con **128 parámetros de 32 bits** porque hay 64 filtros.

7. **`noisy_layer_2`**:
   - Tercera capa convolucional:
     - **36864 pesos binarios (1-bit)**, calculados como: $ 3 \times 3 \times 64 \times 64 = 36864 $.
     - Memoria: **4.50 kB**.

8. **`flatten`**:
   - Aplana la salida para conectarla a capas densas. No tiene parámetros.

9. **`noisy_layer_3`**:
   - Capa densa con 64 unidades:
     - **36864 pesos binarios (1-bit)**, calculados como: $ 576 \times 64 = 36864 $.
     - Memoria: **4.50 kB**.

10. **`noisy_layer_4`**:
    - Capa de salida densa con 10 unidades:
      - **640 pesos binarios (1-bit)**, calculados como: $ 64 \times 10 = 640 $.
      - Memoria: **0.08 kB**.

11. **`activation`**:
    - Usa una función `softmax`. No tiene parámetros.

---

### **2. Resumen general (`+sequential summary-------------------------+`)**
#### **Estadísticas importantes:**
- **Total params**: Número total de parámetros:
  - **93.6 k**: Representa 93088 pesos binarios (1-bit) y 468 parámetros de 32 bits.

- **Trainable params**: Parámetros ajustables durante el entrenamiento:
  - **93.1 k**, que incluye los pesos binarios (1-bit) y parámetros de Batch Normalization.

- **Non-trainable params**: Parámetros que no se ajustan (e.g., estadísticas acumuladas en Batch Normalization):
  - **468**.

- **Model size**:
  - **13.19 KiB**: Tamaño total del modelo en memoria comprimida (pesos binarios).
  - **Model size (8-bit FP weights)**: Si se almacenaran en precisión de 8 bits, ocuparían **11.82 KiB**.
  - **Float-32 Equivalent**: Si todos los pesos fueran flotantes de 32 bits, el modelo ocuparía **365.45 KiB**.

- **Compression Ratio of Memory**:
  - **0.04**: Indica que el modelo es 25 veces más pequeño (4%) en comparación con su equivalente en punto flotante de 32 bits.

- **Number of MACs**:
  - **0**: Esto puede deberse a que el modelo usa operaciones de conteo de bits (binarias) en lugar de acumulaciones/multiplicaciones tradicionales (MAC).

---

### **Conclusión**
La salida refleja que este modelo es **altamente eficiente en términos de memoria**, gracias al uso de **cuantización binaria** para los pesos. Aunque esto reduce la precisión de los cálculos, el modelo sigue siendo compacto y adecuado para dispositivos con recursos limitados, como hardware IoT o ASICs personalizados.

El número de operaciones MAC (multiplicación-acumulación) aparece como **0** porque el modelo usa **cuantización binaria** para los pesos y, posiblemente, para las activaciones. Esto elimina las operaciones tradicionales de multiplicación y acumulación en favor de operaciones mucho más simples, como **suma de conteos de bits** o **XOR**. Veamos por qué y cuál es el impacto de esto:

---

### **1. Por qué el número de MACs es 0**
En modelos binarizados:
- Los pesos son representados en **1-bit**.
- Las multiplicaciones de precisión completa entre activaciones y pesos se reemplazan por:
  - Operaciones binarias, como XOR, para simular multiplicaciones.
  - Sumas de bits resultantes para reemplazar acumulaciones.

Dado que estas operaciones no son multiplicaciones reales en punto flotante o fijo (y Larq optimiza el modelo para hardware binario), no se cuentan como MACs tradicionales.

---

### **2. ¿Tiene sentido modelar las variaciones del proceso CMOS aquí?**
Sí, sigue teniendo sentido añadir las **variaciones del proceso CMOS**, incluso si el modelo no realiza operaciones MAC tradicionales. Esto se debe a que las variaciones del proceso CMOS **pueden influir en el comportamiento de las operaciones binarias y la estabilidad general del hardware**. Aquí hay algunas razones:

#### **Efecto de variaciones CMOS en operaciones binarias:**
1. **Errores en las operaciones XOR/SUMA:**
   - Las variaciones de umbral de voltaje (Vt) pueden afectar la precisión de las operaciones XOR o sumas binarias, introduciendo errores aleatorios.

2. **Ruido en activaciones y pesos binarizados:**
   - Aunque las multiplicaciones completas se eliminan, las activaciones binarizadas y los pesos pueden estar sujetos a variaciones inducidas por CMOS. Esto podría desbalancear los resultados del conteo de bits.

3. **Hardware realista:**
   - Cuando el modelo se implementa en hardware como ASICs o FPGAs, estas variaciones afectan los cálculos a nivel físico, incluso si no hay operaciones MAC tradicionales.

---

### **3. Cómo justificar la inclusión de variaciones CMOS**
Añadir variaciones del proceso CMOS puede ser útil para:
- **Simular hardware realista:** Asegurarte de que el modelo funcione bien en hardware con restricciones prácticas y variaciones físicas.
- **Aumentar la robustez:** Introducir ruido durante el entrenamiento puede hacer que el modelo sea más tolerante a los errores inducidos por hardware.
- **Analizar degradación del rendimiento:** Permitir estudiar cómo las variaciones afectan la precisión general del modelo.

---

### **Conclusión**
Aunque las operaciones MAC no aparecen en modelos binarizados, las variaciones del proceso CMOS afectan otros aspectos del hardware, como las operaciones XOR y el conteo de bits. Por tanto, **sí tiene sentido incluir estas variaciones** para hacer el modelo más robusto y representativo del comportamiento en hardware real. Esto es particularmente relevante si planeas implementarlo en sistemas integrados que usan tecnología CMOS.

In [5]:
# Compile the model with an optimizer and loss function
model_wa.compile(
    optimizer='adam',                      # Adam optimizer is often effective for training binarized networks
    loss="sparse_categorical_crossentropy",# Cross-entropy loss for classification
    metrics=["accuracy"]                   # Accuracy as the evaluation metric
)

# Train the model where QAT takes place as the model learns to optimize quantized weights and activations.
history_wa = model_wa.fit(
    train_images, train_labels,
    epochs=6,                           # Number of training epochs
    batch_size=64,                      # Batch size for training
    validation_data=(test_images, test_labels), # Evaluate on test set after each epoch
    shuffle=False                        # Shuffle the training data for each epoch
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model_wa.evaluate(test_images, test_labels)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Epoch 1/6
938/938 [==============================] - 118s 123ms/step - loss: 1.0993 - accuracy: 0.7421 - val_loss: 1.2276 - val_accuracy: 0.7237
Epoch 2/6
938/938 [==============================] - 105s 112ms/step - loss: 0.8363 - accuracy: 0.8550 - val_loss: 1.0061 - val_accuracy: 0.8196
Epoch 3/6
938/938 [==============================] - 105s 112ms/step - loss: 0.7155 - accuracy: 0.8958 - val_loss: 0.7922 - val_accuracy: 0.8881
Epoch 4/6
938/938 [==============================] - 102s 108ms/step - loss: 0.6504 - accuracy: 0.9136 - val_loss: 0.7438 - val_accuracy: 0.9122
Epoch 5/6
938/938 [==============================] - 105s 112ms/step - loss: 0.6154 - accuracy: 0.9258 - val_loss: 0.6766 - val_accuracy: 0.9259
Epoch 6/6
313/313 [==============================] - 9s 28ms/step - loss: 0.7649 - accuracy: 0.9168
Test Accuracy: 91.68%
Test Loss: 0.7649


## 4. VARIACIONES DE PROCESO CMOS

Las variaciones de proceso pueden cambiar los parámetros de los transistores, afectando el comportamiento de las operaciones aritméticas. Esto puede modelarse como una desviación en los pesos del modelo.

### **Análisis del Código Actual**

hemos definido un modelo de red neuronal convolucional en **TensorFlow** con capas cuantizadas usando **Larq**. Además, hemos implementado una capa personalizada (`NoisyLayer`) que introduce ruido gaussiano en los pesos y activaciones de otras capas para simular imprecisiones o ruido en el hardware en un modelo CNN llamado `model_wa` (**noise in weights and activations**).

#### **Características**
1. **Capa personalizada (`NoisyLayer`)**:
   - **Pesos**: Introduce ruido gaussiano en los pesos de la capa antes de la operación `call`.
   - **Activaciones**: Añade ruido gaussiano a las salidas de la capa.

2. **Modelo secuencial (`model_wa`)**:
   - Se utiliza ruido en los datos de entrada/imagen (`GaussianNoise`).
   - Cada capa convolucional y densa está envuelta (wrapped) con `NoisyLayer` para simular hardware ruidoso.
   - Usamos Larq (`lq`) para implementar las capas cuantizadas para QAT.

3. **Finalidad del diseño**: Simular entornos con hardware real. El modelo es lo suficiente robusto frente al ruido inducido.

---

### **Introducir Variaciones del Proceso CMOS**

El comportamiento de un circuito CMOS puede ser afectado por **variaciones de proceso**, que incluyen:
- **Threshold Voltage Variation (Vt)**: Cambios en el umbral de los transistores afectan operaciones lógicas y de cuantización.
- **Capacitive Coupling Noise**: Introduce inexactitudes adicionales.
- **Temperature-induced Drift**: Cambios estocásticos en la precisión debido a temperatura.

Para modelar estas variaciones, podemos extender el comportamiento del `NoisyLayer` para incluir:
1. **Imprecisiones en operaciones aritméticas específicas** (e.g., multiplicación, suma).
2. **Sesgo adicional en el ruido de pesos y activaciones basado en drift térmico o variación de Vt**.


### **Explicación de Modificaciones**
1. **Variación de Voltaje de Umbral (Vt)**:
   - Añadimos un término multiplicativo `1 + vt_variation` en los pesos para modelar el efecto de variaciones estocásticas del proceso CMOS.

2. **Ruido por Acoplamiento Capacitivo**:
   - Introducimos un ruido gaussiano adicional (`capacitive_noise_stddev`) en las activaciones para reflejar la interferencia de acoplamiento.

3. **Nueva Clase (`CMOSNoisyLayer`)**:
   - Esta capa personalizada es una extensión de `NoisyLayer` que incluye las variaciones mencionadas.

---

Este modelo pretende ser `más realista` para simulaciones en hardware CMOS, ya que incorpora características específicas del proceso de fabricación y las operaciones aritméticas.


In [6]:
import larq as lq
import tensorflow as tf

# Define a custom layer to add CMOS process variation effects
class CMOSNoisyLayer(tf.keras.layers.Layer):
    def __init__(
        self,
        wrapped_layer,
        noise_mean=0.0,
        noise_stddev_weights=0.01,
        noise_stddev_activations=0.01,
        vt_variation_factor=0.001,
        capacitive_noise_stddev=0.005,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.wrapped_layer = wrapped_layer                 # Original layer to wrap
        self.noise_mean = noise_mean                       # Mean of the noise
        self.noise_stddev_weights = noise_stddev_weights   # Stddev of noise for weights
        self.noise_stddev_activations = noise_stddev_activations  # Stddev of noise for activations
        self.vt_variation_factor = vt_variation_factor     # Threshold voltage variation factor
        self.capacitive_noise_stddev = capacitive_noise_stddev  # Capacitive coupling noise

    def build(self, input_shape):
        self.wrapped_layer.build(input_shape)
        super().build(input_shape)

    def call(self, inputs):
        # Add noise to weights (including CMOS process variations)
        for weight in self.wrapped_layer.trainable_weights:
            noise = tf.random.normal(
                shape=tf.shape(weight),
                mean=self.noise_mean,
                stddev=self.noise_stddev_weights
            )
            vt_variation = tf.random.uniform(
                shape=tf.shape(weight),
                minval=-self.vt_variation_factor,
                maxval=self.vt_variation_factor
            )
            noisy_weight = weight * (1 + vt_variation) + noise  # Combine variations
            weight.assign(noisy_weight)  # Assign noisy weights temporarily

        # Forward pass through the wrapped layer
        outputs = self.wrapped_layer(inputs)

        # Add noise to activations
        activation_noise = tf.random.normal(
            shape=tf.shape(outputs),
            mean=self.noise_mean,
            stddev=self.noise_stddev_activations
        )
        capacitive_noise = tf.random.normal(
            shape=tf.shape(outputs),
            mean=0.0,
            stddev=self.capacitive_noise_stddev
        )
        outputs_noisy = outputs + activation_noise + capacitive_noise  # Add combined noise

        return outputs_noisy


# Define default quantization options for quantized layers
kwargs = dict(
    input_quantizer="ste_sign",
    kernel_quantizer="ste_sign",
    kernel_constraint="weight_clip"
)

# Initialize a Sequential model
model_wa_cmos = tf.keras.models.Sequential()

# Add noise to input images (simulate noisy input images)
model_wa_cmos.add(tf.keras.layers.GaussianNoise(0.05, input_shape=(28, 28, 1)))

# First convolutional layer with CMOS process noise
model_wa_cmos.add(CMOSNoisyLayer(
    lq.layers.QuantConv2D(
        32, (3, 3),
        kernel_quantizer="ste_sign",
        kernel_constraint="weight_clip",
        use_bias=False
    ),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01,
    vt_variation_factor=0.002,
    capacitive_noise_stddev=0.005
))
model_wa_cmos.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_wa_cmos.add(tf.keras.layers.BatchNormalization(scale=False))

# Second convolutional layer
model_wa_cmos.add(CMOSNoisyLayer(
    lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01,
    vt_variation_factor=0.002,
    capacitive_noise_stddev=0.005
))
model_wa_cmos.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_wa_cmos.add(tf.keras.layers.BatchNormalization(scale=False))

# Third convolutional layer
model_wa_cmos.add(CMOSNoisyLayer(
    lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01,
    vt_variation_factor=0.002,
    capacitive_noise_stddev=0.005
))
model_wa_cmos.add(tf.keras.layers.BatchNormalization(scale=False))
model_wa_cmos.add(tf.keras.layers.Flatten())

# Dense layer
model_wa_cmos.add(CMOSNoisyLayer(
    lq.layers.QuantDense(64, use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01,
    vt_variation_factor=0.002,
    capacitive_noise_stddev=0.005
))
model_wa_cmos.add(tf.keras.layers.BatchNormalization(scale=False))

# Output layer
model_wa_cmos.add(CMOSNoisyLayer(
    lq.layers.QuantDense(10, use_bias=False, **kwargs),
    noise_stddev_weights=0.01,
    noise_stddev_activations=0.01,
    vt_variation_factor=0.002,
    capacitive_noise_stddev=0.005
))
model_wa_cmos.add(tf.keras.layers.BatchNormalization(scale=False))
model_wa_cmos.add(tf.keras.layers.Activation("softmax"))

# Especificar la forma de entrada (28, 28, 1) para imágenes MNIST
model_wa_cmos.build((None, 28, 28, 1))

# Summarize the model with Larq
lq.models.summary(model_wa)

+sequential stats-----------------------------------------------------------------+
| Layer                  Input prec.           Outputs  # 1-bit  # 32-bit  Memory |
|                              (bit)                        x 1       x 1    (kB) |
+---------------------------------------------------------------------------------+
| gaussian_noise                   -   (-1, 28, 28, 1)        0         0       0 |
| noisy_layer                      -  (-1, 26, 26, 32)      288         0    0.04 |
| max_pooling2d                    -  (-1, 13, 13, 32)        0         0       0 |
| batch_normalization              -  (-1, 13, 13, 32)        0        64    0.25 |
| noisy_layer_1                    -  (-1, 11, 11, 64)    18432         0    2.25 |
| max_pooling2d_1                  -    (-1, 5, 5, 64)        0         0       0 |
| batch_normalization_1            -    (-1, 5, 5, 64)        0       128    0.50 |
| noisy_layer_2                    -    (-1, 3, 3, 64)    36864         0   

In [7]:
# Compile the model with an optimizer and loss function
model_wa_cmos.compile(
    optimizer='adam',                      # Adam optimizer is often effective for training binarized networks
    loss="sparse_categorical_crossentropy",# Cross-entropy loss for classification
    metrics=["accuracy"]                   # Accuracy as the evaluation metric
)

# Train the model where QAT takes place as the model learns to optimize quantized weights and activations.
history_wa_cmos = model_wa_cmos.fit(
    train_images, train_labels,
    epochs=6,                           # Number of training epochs
    batch_size=64,                      # Batch size for training
    validation_data=(test_images, test_labels), # Evaluate on test set after each epoch
    shuffle=False                        # Shuffle the training data for each epoch
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model_wa_cmos.evaluate(test_images, test_labels)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

Epoch 1/6
938/938 [==============================] - 141s 148ms/step - loss: 1.1081 - accuracy: 0.7367 - val_loss: 1.2098 - val_accuracy: 0.7240
Epoch 2/6
938/938 [==============================] - 142s 152ms/step - loss: 0.8231 - accuracy: 0.8588 - val_loss: 0.9427 - val_accuracy: 0.8506
Epoch 3/6
938/938 [==============================] - 146s 155ms/step - loss: 0.7091 - accuracy: 0.8965 - val_loss: 0.8577 - val_accuracy: 0.8602
Epoch 4/6
938/938 [==============================] - 139s 148ms/step - loss: 0.6483 - accuracy: 0.9160 - val_loss: 0.8757 - val_accuracy: 0.8646
Epoch 5/6
938/938 [==============================] - 141s 150ms/step - loss: 0.6076 - accuracy: 0.9297 - val_loss: 0.6792 - val_accuracy: 0.9230
Epoch 6/6
313/313 [==============================] - 18s 58ms/step - loss: 0.8918 - accuracy: 0.8815
Test Accuracy: 88.15%
Test Loss: 0.8918
